# Google Cloud Dataflow

Table of contents:

- [Beam Programming](#beam-programming)

- Google Cloud Dataflow provides a serverless, parallel and distributed infrastructure for running jobs for batch and stream data processing.
- One of the core strengths of Dataflow is its ability to almost seamlessly handle the switch from processing of batch historical data to streaming datasets while elegantly taking into consideration the perks of streaming processing such as windowing.
- Dataflow is a major component of the Data/ML pipeline on GCP.

<a id="beam-programming"></a>

## Beam Programming
Apache Beam provides a set of broad concepts to simplify the process of building a transformation pipeline for distributed batch and stream jobs.

- **A Pipeline:** A Pipeline object wraps the entire operation and prescribes the transformation process by defining the input data source to the pipeline, how that data will be transformed and where the data will be written.
- **A PCollection:** A PCollection is used to define a data source. The data source can either be bounded or unbounded. A bounded data source referes to batch or historical data, whereas an unbounded data source refers to streaming data.
- **A PTransform:** PTransforms refers to a particular transformation task carried out on one or more PCollections in the pipeline. A number of core Beam transforms include:
  - ParDo: for parallel processing.
  - GroupByKey: for processing collections of key/value pairs.
  - CoGroupByKey: for a relational join of two or more key/value PCollections with the same key type.
  - Combine: for combining collections of elements or values in your data.
  - Flatten: for merging multiple PCollection objects.
  - Partition: splits a single PCollection into smaller collections. 
- **I/O Transforms:** These are PTransforms that read or write data to different external storage systems.

<div style="display: inline-block;width: 100%;">
<img src="ieee-ompi/brain.png" style="float:left;" alt="A Neuron." height=90% width=90% />
</div>